In [1]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
import testbench
import chess_utils
import numpy as np

def games_to_opening_vecs(games):
    return np.array(list(map(lambda game: chess_utils.game_to_vec(game, 35), games)))

nn_model = MLPRegressor(hidden_layer_sizes=(128, 64, 32), activation='relu', solver='adam', max_iter=500)

nn_pipe = Pipeline([
    ('Game to vec', FunctionTransformer(games_to_opening_vecs)),
    ('Neural Network', nn_model)
])

# 2 hyperparams to tune: # moves considered in game2vec; Neural network architecture

testbench.test(nn_pipe, 50000, 10000, 'big_nn_report', 'Neural network final performance test.')


{'Description': 'Neural network final performance test.', 'Pipeline': "{'Game to vec': FunctionTransformer(func=<function games_to_opening_vecs at 0x0000022DCEB88CC0>), 'Neural Network': MLPRegressor(hidden_layer_sizes=(128, 64, 32), max_iter=500)}", '# Games for training': 50000, '# Games for testing': 10000, 'Train Fit time': 7636.6940331459045, 'Train R2 score': 0.9154532690959276, 'Train MSE': np.float64(5834.1417631537715), 'Test Predict time': 13.79459810256958, 'Test R2 score': -0.24961000107378983, 'Test MSE': np.float64(89690.15210868108)}


In [2]:
import joblib

# Save the trained pipeline
joblib.dump(nn_pipe, 'models/nn_pipeline1.pkl')
joblib.dump(nn_model, 'models/nn_model1.pkl')
print("Model saved successfully!")

Model saved successfully!


In [3]:
import chess.pgn
import io
import numpy as np

# The given PGN game
pgn_text = """[Event "FICS rated standard game"]
[Site "FICS freechess.org"]
[Date "2020.06.30"]
[Round "?"]
[White "crashoverriiide"]
[Black "YavChess"]
[Result "1-0"]
[BlackClock "0:15:00.000"]
[BlackElo "1859"]
[BlackRD "0.0"]
[ECO "B27"]
[FICSGamesDBGameNo "472656335"]
[PlyCount "111"]
[Time "23:29:00"]
[TimeControl "900+0"]
[WhiteClock "0:15:00.000"]
[WhiteElo "1652"]
[WhiteRD "0.0"]

1. e4 c5 2. Nf3 g6 3. Bc4 Bg7 4. c3 Nc6 5. O-O e6 6. Re1 Nge7 7. Bb3 O-O 8. d3 b5 9. Bg5 Bb7 10. Qd2 Re8 11. Bh6 Bh8 12. Bg5 Qc7 13. Qe3 Na5 14. Bc2 d5 15. Bxe7 Rxe7 16. d4 cxd4 17. Nxd4 Nc4 18. Qc1 a6 19. Nd2 Rc8 20. N2f3 Ne5 21. a3 Nd3 22. Bxd3 e5 23. Nc2 Rd7 24. Qh6 Bg7 25. Qh3 h6 26. Rad1 dxe4 27. Bxe4 Rxd1 28. Rxd1 Bxe4 29. Ne3 Rd8 30. Re1 Qc6 31. Qg4 Bxf3 32. gxf3 h5 33. Qg3 Rd3 34. Rd1 Rxd1+ 35. Nxd1 Qd5 36. Ne3 Qd2 37. Qg5 Kh7 38. Qe7 f5 39. Qb4 f4 40. Nf1 Qd5 41. Qe4 Qb3 42. Qb1 Qc4 43. Nd2 Qe2 44. Qc1 Bf8 45. Kg2 Bc5 46. Ne4 Be3 47. Qf1 Qxf1+ 48. Kxf1 Bc1 49. b3 Bxa3 50. Nf6+ Kh8 51. Nd7 Bd6 52. Nb6 Bc5 53. Nd5 a5 54. b4 a4 55. bxc5 Kg7 56. c6 { Black forfeits on time } 1-0"""

# Parse the PGN
pgn_io = io.StringIO(pgn_text)
game = chess.pgn.read_game(pgn_io)

# Convert a single game to a feature vector
game_vector = chess_utils.game_to_vec(game, 35).reshape(1, -1)

# Predict Elo directly using the kNN model (not the pipeline)
predicted_elo = nn_model.predict(game_vector)

print(predicted_elo.shape)

white_elo, black_elo = predicted_elo.flatten()
print(f"Predicted White Elo: {white_elo:.2f}")
print(f"Predicted Black Elo: {black_elo:.2f}")


(1, 2)
Predicted White Elo: 1795.68
Predicted Black Elo: 1792.29
